# Stock price collector

In [4]:
from pandas_datareader import data
from pandas_datareader._utils import RemoteDataError
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import csv
import re
from datetime import datetime

import os.path
from os import path
import time

# import sklearn
# import scipy

# from sklearn.naive_bayes import GaussianNB
# import lightgbm as lightgbm
# from xgboost import XGBClassifier
# import keras

# Get Started
# https://github.com/Refinitiv-API-Samples/Article.EikonDataAPI.DotNet.Library
# https://developers.refinitiv.com/en/api-catalog/eikon/eikon-data-api/quick-start
# API limits
# https://developers.refinitiv.com/en/api-catalog/eikon/eikon-data-api/documentation?content=49692&type=documentation_item
import eikon as ek 

from pylab import rcParams
plt.rcParams['figure.figsize'] = 16, 8

In [5]:
fontsize_reg = 12
fontsize_title = 16
from pylab import rcParams
rcParams['figure.facecolor'] = '1'
rcParams['figure.figsize'] = [8.0, 3.5]
rcParams['figure.dpi'] = 80
rcParams['savefig.dpi'] = 600

rcParams['font.size'] = 12
rcParams['legend.fontsize'] = 'large'
rcParams['figure.titlesize'] = 'large'

In [6]:
# Зададим path для папки проекта и для папок файлов
path_project_folder = 'C:/DAN/t_systems/trade_project/'
folder_name_headlines = 'backup_headlines/'
folder_name_logger = 'logger/'
folder_name_price_timeseries = 'price_timeseries/'

In [7]:
# Для работы с API нужен запущенный терминал Thomson Reuters Eikon
# Зададим API ключ - не даёт доступ сам по себе
ek.set_app_key('df6c94f7d0fb4b4e80533d1c2f86daf402ce52c0') 
# ccebd1ff788d4574818be93e35c4741925ac214e
# df6c94f7d0fb4b4e80533d1c2f86daf402ce52c0

In [21]:
df = ek.get_timeseries(
    ["MSFT.O"],
    interval='minute',
    start_date="2010-01-01",
    end_date="2022-01-30",
    fields=['TIMESTAMP', 'VALUE', 'VOLUME', 'HIGH', 'LOW', 'OPEN', 'CLOSE', 'COUNT'],
)

df

2021-03-13 15:46:15,901 P[14904] [MainThread 12212] Error code 429 | Client Error: Too many requests, please try again later. Wait for 29625 seconds.
2021-03-13 15:46:15,903 P[14904] [MainThread 12212] HTTP request failed: EikonError-Client Error: Too many requests, please try again later. Wait for 29625 seconds.


In [9]:
def get_name_of_latest_file_in_folder(folder_name):
    """
    Get all file names in selected direction.
    Return the latest file in that directory.
    """
    files_creation_dates = pd.DataFrame(data = [], columns=[
        "folder_name", "file_name", "num_time", "str_time"
    ])

    files_creation_dates['file_name'] = os.listdir(folder_name)
    files_creation_dates['folder_name'] = folder_name
    files_creation_dates['file_folder_and_name'] = files_creation_dates['folder_name'] + files_creation_dates['file_name']
    files_creation_dates
    for i in range(0, len(files_creation_dates)):
        file_folder_and_name = files_creation_dates['file_folder_and_name'][i]
        files_creation_dates['num_time'][i] = os.path.getctime(file_folder_and_name)
        files_creation_dates['str_time'][i] = time.ctime(os.path.getctime(file_folder_and_name))

    latest_file_dir = files_creation_dates.sort_values(by=['num_time'], ascending=False)['file_folder_and_name'].iloc[0]
    
    return latest_file_dir

In [10]:
# Read the file with all news texts to create a list with rics in it
latest_file = get_name_of_latest_file_in_folder(path_project_folder + folder_name_headlines)
all_headlines_df = pd.read_csv(latest_file, sep=';')
all_headlines_df['versionCreated'] = pd.to_datetime(all_headlines_df['versionCreated'])
print(all_headlines_df.shape, latest_file)

(389246, 6) C:/DAN/t_systems/trade_project/backup_headlines/all_headlines_df_2021-03-11_v5.csv


In [11]:
# Create the file with rics from all news texts dataset
rics_list = list(all_headlines_df['ric'].value_counts().index)
print(len(rics_list), rics_list)

# rics_list = ['JPM.N', 'AMZN.OQ', 'FB.OQ', 'AAPL.OQ', 'MSFT.OQ', 'GOOGL.OQ', 'TSLA.OQ', 'BABA.N', 'PFE.N', 'RWEG.DE', 'VOWG_p.DE', 'BAC.N', 'NFLX.OQ', 'WMT.N', 'SIEGn.DE', 'LHAG.DE', 'DBKGn.DE', 'DIS.N', 'JNJ.N', 'INTC.OQ', 'DAIGn.DE', 'T.N', 'BMWG.DE', 'MRK.N', 'BAYGn.DE', 'VZ.N', 'SAPG.DE', 'EONGn.DE', 'KO.N', 'CBKG.DE', 'MA.N', 'ALVG.DE', 'NKE.N', 'TKAG.DE', 'DTEGn.DE', 'NVDA.OQ', 'V.N', 'CMCSA.OQ', 'BRKa.N', 'UNH.N', 'BASFn.DE', 'PG.N', 'HD.N', 'TMUS.OQ', 'PYPL.OQ', 'ADSGn.DE', 'DPWGn.DE', 'CONG.DE', 'LUMN.N', 'PSMGn.DE', 'MRCG.DE', 'IFXGn.DE', 'FREG.DE', 'HNKG_p.DE', 'FMEG.DE', 'ATUS.N', 'ADBE.OQ', 'CHTR.OQ', 'VNAn.DE', 'JCOM.OQ', 'HEIG.DE', 'O2Dn.DE', 'PDD.OQ', 'UTDI.DE', 'VG.OQ', 'FNTGn.DE', 'BEIG.DE', 'DRIG.DE', 'TIGO.OQ', 'BAND.OQ', 'TC1n.DE', 'IRDM.OQ', 'CABO.N', 'CCOI.OQ', 'USM.N', 'NFN.DE', 'E4CG.DE', 'PH6.F', 'IVV.P', 'VTI.P']

85 ['JPM.N', 'AMZN.OQ', 'FB.OQ', 'AAPL.OQ', 'MSFT.OQ', 'GOOGL.OQ', 'TSLA.OQ', 'BABA.N', 'PFE.N', 'RWEG.DE', 'VOWG_p.DE', 'BAC.N', 'NFLX.OQ', 'WMT.N', 'SIEGn.DE', 'LHAG.DE', 'DBKGn.DE', 'DIS.N', 'JNJ.N', 'INTC.OQ', 'DAIGn.DE', 'T.N', 'BMWG.DE', 'MRK.N', 'BAYGn.DE', 'VZ.N', 'SAPG.DE', 'EONGn.DE', 'CRM.N', 'KO.N', 'CBKG.DE', 'MA.N', 'ALVG.DE', 'NKE.N', 'TKAG.DE', 'DTEGn.DE', 'XOM.N', 'NVDA.OQ', 'V.N', 'CMCSA.OQ', 'BRKa.N', 'UNH.N', 'BASFn.DE', 'PG.N', 'HD.N', 'TMUS.OQ', 'ABT.N', 'PYPL.OQ', 'PEP.OQ', 'TMO.N', 'ADSGn.DE', 'DPWGn.DE', 'CONG.DE', 'LUMN.N', 'PSMGn.DE', 'MRCG.DE', 'IFXGn.DE', 'FREG.DE', 'HNKG_p.DE', 'FMEG.DE', 'ATUS.N', 'ADBE.OQ', 'CHTR.OQ', 'VNAn.DE', 'JCOM.OQ', 'HEIG.DE', 'O2Dn.DE', 'PDD.OQ', 'UTDI.DE', 'VG.OQ', 'FNTGn.DE', 'BEIG.DE', 'DRIG.DE', 'TIGO.OQ', 'BAND.OQ', 'TC1n.DE', 'IRDM.OQ', 'CABO.N', 'CCOI.OQ', 'USM.N', 'NFN.DE', 'E4CG.DE', 'PH6.F', 'IVV.P', 'VTI.P']


In [12]:
import time

def get_timeseries_of_rics_to_folder(
    list_of_rics,
    folder_for_timeseries_file=path_project_folder+folder_name_price_timeseries,
    timeseries_interval=str('daily'),
    timeseries_start_date=str("2010-01-01"),
    timeseries_end_date=str("2021-06-01"),
    timeseries_is_adjusted=True,
    sleep_between_rics=0):
    
    for ric_now in list_of_rics:
        index_of_ric_now = list_of_rics.index(ric_now)
        
        print(index_of_ric_now, ric_now, end=', ')
        
        # Try to get the data via API untill the number of attempts is over
        timeseries_df = None
        number_of_attempts=1
        while (timeseries_df is None) and (number_of_attempts < 3):
            try:
                # Get df via Eikon Data API
                # Possible values of interval: 
                # 'tick', 'minute', 'hour', 
                # 'daily', 'weekly', 'monthly', 'quarterly', 'yearly' (Default 'daily')
                corax_param = 'adjusted' if timeseries_is_adjusted else 'unadjusted'
                timeseries_df = ek.get_timeseries(
                    [ric_now],
                    interval=timeseries_interval,
                    start_date=timeseries_start_date,
                    end_date=timeseries_end_date,
                    fields=['TIMESTAMP', 'VALUE', 'VOLUME', 'HIGH', 'LOW', 'OPEN', 'CLOSE', 'COUNT'],
                    corax=corax_param
                )
            except Exception:
                print('ERROR, Exception. Request fails or if server returns an error.')
                time.sleep(5)
                pass
            except ValueError:
                print('ERROR, ValueError. A parameter type or value is wrong.')
                time.sleep(5)
                pass
            
            print('Attempt', number_of_attempts, end=', ')
            
            if timeseries_df is None: # ERROR
                print('error with that attempt', end=', ')
                time.sleep(60)
            number_of_attempts = number_of_attempts + 1
            
        # End of while
        
        # If it fail several attempts then print the final error message
        if timeseries_df.columns.name == None:
            print(
                'ERROR WITH THIS RIC:', 
                ric_now, 
                timeseries_interval, 
                timeseries_start_date, 
                timeseries_end_date,
                corax_param
            )
            pass
        else:
            print('NOT EMPTY', end=', ')
        
        # Add ric and date columns
        timeseries_df['ric'] = timeseries_df.columns.name 
        # Check index type to do not reset_index again
        if type(timeseries_df.index[0]) != int:
            timeseries_df['Date'] = timeseries_df.index
            timeseries_df = timeseries_df.reset_index(drop=True, inplace=False)

        # Save created df with one ric. File name contains ric name
        file_name = 'price_timeseries' + '_' + timeseries_interval + '_' + corax_param + '_' + timeseries_df.columns.name
        file_path =  folder_for_timeseries_file + file_name + '.csv'
        timeseries_df.to_csv(file_path, sep = ';', index=False)
        print()
        print(' '*(len(ric_now) + len(str(index_of_ric_now))), ', ', end='')
        print(file_path, end=', ')
        
        print(datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
        
        if sleep_between_rics > 0:
            time.sleep(sleep_between_rics)

In [13]:
rics_list[35]

'DTEGn.DE'

In [20]:
get_timeseries_of_rics_to_folder(
    list_of_rics=rics_list,
    folder_for_timeseries_file=path_project_folder+folder_name_price_timeseries,
    timeseries_interval='daily',
    timeseries_start_date="2010-01-01",
    timeseries_end_date="2021-06-01",
    timeseries_is_adjusted=True,
    sleep_between_rics=2)

0 JPM.N, Attempt 1, NOT EMPTY, 
       , C:/DAN/t_systems/trade_project/price_timeseries/price_timeseries_daily_adjusted_JPM.N.csv, 12/03/2021 18:39:57
1 AMZN.OQ, Attempt 1, NOT EMPTY, 
         , C:/DAN/t_systems/trade_project/price_timeseries/price_timeseries_daily_adjusted_AMZN.OQ.csv, 12/03/2021 18:40:00
2 FB.OQ, Attempt 1, NOT EMPTY, 
       , C:/DAN/t_systems/trade_project/price_timeseries/price_timeseries_daily_adjusted_FB.OQ.csv, 12/03/2021 18:40:04
3 AAPL.OQ, Attempt 1, NOT EMPTY, 
         , C:/DAN/t_systems/trade_project/price_timeseries/price_timeseries_daily_adjusted_AAPL.OQ.csv, 12/03/2021 18:40:08
4 MSFT.OQ, Attempt 1, NOT EMPTY, 
         , C:/DAN/t_systems/trade_project/price_timeseries/price_timeseries_daily_adjusted_MSFT.OQ.csv, 12/03/2021 18:40:15
5 GOOGL.OQ, Attempt 1, NOT EMPTY, 
          , C:/DAN/t_systems/trade_project/price_timeseries/price_timeseries_daily_adjusted_GOOGL.OQ.csv, 12/03/2021 18:40:19
6 TSLA.OQ, Attempt 1, NOT EMPTY, 
         , C:/DAN/t_systems

In [14]:
list_of_files_to_merge = os.listdir(path_project_folder + folder_name_price_timeseries)
print(list_of_files_to_merge)

['price_timeseries_daily_adjusted_AAPL.OQ.csv', 'price_timeseries_daily_adjusted_ALVG.DE.csv', 'price_timeseries_daily_adjusted_AMZN.OQ.csv', 'price_timeseries_daily_adjusted_BABA.N.csv', 'price_timeseries_daily_adjusted_BAC.N.csv', 'price_timeseries_daily_adjusted_BAYGn.DE.csv', 'price_timeseries_daily_adjusted_BMWG.DE.csv', 'price_timeseries_daily_adjusted_CBKG.DE.csv', 'price_timeseries_daily_adjusted_DAIGn.DE.csv', 'price_timeseries_daily_adjusted_DBKGn.DE.csv', 'price_timeseries_daily_adjusted_DIS.N.csv', 'price_timeseries_daily_adjusted_DTEGn.DE.csv', 'price_timeseries_daily_adjusted_EONGn.DE.csv', 'price_timeseries_daily_adjusted_FB.OQ.csv', 'price_timeseries_daily_adjusted_GOOGL.OQ.csv', 'price_timeseries_daily_adjusted_INTC.OQ.csv', 'price_timeseries_daily_adjusted_JNJ.N.csv', 'price_timeseries_daily_adjusted_JPM.N.csv', 'price_timeseries_daily_adjusted_KO.N.csv', 'price_timeseries_daily_adjusted_LHAG.DE.csv', 'price_timeseries_daily_adjusted_MA.N.csv', 'price_timeseries_daily

In [15]:
all_stock_prices = pd.DataFrame(
    data = [], 
    columns=[])

for file_now in list_of_files_to_merge:
    file_path = path_project_folder + folder_name_price_timeseries + file_now
    timeseries_df = pd.read_csv(file_path, sep=';')
    print(file_path, timeseries_df.shape)
    
    all_stock_prices = pd.concat([all_stock_prices, timeseries_df], ignore_index=True)
    time.sleep(0)

C:/DAN/t_systems/trade_project/price_timeseries/price_timeseries_daily_adjusted_AAPL.OQ.csv (2810, 9)
C:/DAN/t_systems/trade_project/price_timeseries/price_timeseries_daily_adjusted_ALVG.DE.csv (2831, 9)
C:/DAN/t_systems/trade_project/price_timeseries/price_timeseries_daily_adjusted_AMZN.OQ.csv (2810, 9)
C:/DAN/t_systems/trade_project/price_timeseries/price_timeseries_daily_adjusted_BABA.N.csv (1624, 9)
C:/DAN/t_systems/trade_project/price_timeseries/price_timeseries_daily_adjusted_BAC.N.csv (2810, 9)
C:/DAN/t_systems/trade_project/price_timeseries/price_timeseries_daily_adjusted_BAYGn.DE.csv (2831, 9)
C:/DAN/t_systems/trade_project/price_timeseries/price_timeseries_daily_adjusted_BMWG.DE.csv (2831, 9)
C:/DAN/t_systems/trade_project/price_timeseries/price_timeseries_daily_adjusted_CBKG.DE.csv (2831, 9)
C:/DAN/t_systems/trade_project/price_timeseries/price_timeseries_daily_adjusted_DAIGn.DE.csv (2831, 9)
C:/DAN/t_systems/trade_project/price_timeseries/price_timeseries_daily_adjusted_DBK

In [18]:
all_stock_prices

,VALUE,VOLUME,HIGH,LOW,OPEN,CLOSE,COUNT,ric,Date
0,NaN,1.525399e+08,7.660707,7.586421,7.626778,7.649278,-1,AAPL.OQ,2010-01-04
1,NaN,1.704628e+08,7.699635,7.616421,7.671064,7.656421,-1,AAPL.OQ,2010-01-05
2,NaN,1.879475e+08,7.685707,7.526778,7.655707,7.534635,-1,AAPL.OQ,2010-01-06
3,NaN,1.456166e+08,7.571421,7.466421,7.559992,7.520707,-1,AAPL.OQ,2010-01-07
4,NaN,1.410746e+08,7.571421,7.466421,7.517492,7.570707,-1,AAPL.OQ,2010-01-08
...,...,...,...,...,...,...,...,...,...
99542,NaN,2.960677e+06,134.000000,131.500000,133.000000,131.950000,17001,WMT.N,2021-02-25
99543,NaN,4.073071e+06,131.750000,129.870000,131.480000,129.920000,18833,WMT.N,2021-02-26
99544,NaN,2.349535e+06,132.570000,131.150000,131.580000,131.370000,13085,WMT.N,2021-03-01
99545,NaN,2.225216e+06,132.720000,130.010000,131.510000,130.110000,11962,WMT.N,2021-03-02
